# 1.閒聊問答實作

In [ ]:
# 請將 "1-1.自定義 Q&A" 的程式碼修改如下

In [ ]:
from fuzzychinese import FuzzyChineseMatch

greating = "你是誰"
to_girl = [("請管好你的嘴","為什麼","因為我會隨時親你"),
           ("你單身多久了","一年多了","對不起，讓你等了這麼久"),
           ("我真的是太笨了","怎麼啦","除了喜歡你，其它什麼都做不好")]
similars =  "最近有什麼好看的電影 推薦不錯的電影 那些影片評價不錯 高評價的電影 很棒的影片".split(" ")

def fuzzy_mapping(query, similars, threshold=0.7):
    fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='stroke')
    fcm.fit(similars)
    most_similar = fcm.transform([query], n=1)
    most_similar_score = round(float(fcm.get_similarity_score()[0][0]),3)   
    print(most_similar_score)
    if most_similar_score >= threshold:
        return True
    else:
        return False

def get_cutomize_ans(query):
    # 完全比對
    if query == greating:
        return "我是你的助理! 我可以陪你聊天喔~"

    # 部分比對
    elif "撩妹" in query:
        return str(random.choice(to_girl))

    # 模糊比對-1
    elif "台北" in query and "景點" in query:
        return "有101, 中正紀念堂還可以去爬象山喔!"
    
    elif fuzzy_mapping(query, similars, threshold=0.7):
        return "最近當然要看鬼滅劇場版阿!"
    
    elif any(i in query for i in "台北 台北市 台灣首都 天龍國".split()):
        if any(j in query for j in "手搖飲料店 珍奶店 飲料店".split()):
            url = "https://google.com/search?hl=zh-TW&q=台北市手搖飲料店&num=10"
            return url
        elif any(j in query for j in "名產 伴手禮 土產".split()):
            url = "https://google.com/search?hl=zh-TW&q=台北市名產&num=10"
            return url
        
        else:
            return None
            # return "我還沒辦法回覆你的問題喔!"
    else:
        return None
        # return "我還沒辦法回覆你的問題喔!"

ModuleNotFoundError: ignored

# 2.知識問答實作

In [ ]:
# 請將 "2.知識問答實作" 的程式碼修改如下

In [ ]:
import jieba
import jieba.posseg
import urllib.parse
import requests, re
from bs4 import BeautifulSoup

check_list = ["nr", "nz", "nrt"]


# 進行斷詞
def jieba_tokenizer(query):
    word_pos = jieba.posseg.cut(query)
    return [(word,pos) for word, pos in word_pos]

# 人名判斷
def pos_checker(check_list, token):
    for w, p in token:
        if any( i in p for i in check_list):
            return True
    return False

# 取得 html   
def get_html(query, num):
    query = urllib.parse.quote_plus(query)
    url = 'https://google.com/search?hl=zh-TW&q=' + query + "&num=" + str(num)
    try:
        response = requests.get(url)
        return BeautifulSoup(response.text, 'lxml')
    except:
        return None
    
# 取得搜尋結果的標題
def get_title(text):
    title = re.findall(r'<h3 class="zBAuLc"><div class="BNeawe vvjwJb AP7Wnd">(?P<content>.*?)</div></h3>', str(text), re.DOTALL)
    if title != []:
        title = title[0].replace('\n', '')
    return title

# 取得結果摘要
def get_content(text):
    content = re.findall(r'<div class="kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd"><div><div><div class="BNeawe s3v9rd AP7Wnd">(?P<content>.*?)</div></div></div></div></div></div></div>', str(text), re.DOTALL)
    if content != []:
        content = content[0].replace('\n', '')
        content = re.sub(u'<span class="r0bn4c rQMQod">.*?"r0bn4c rQMQod"> · </span>', "", content)
    return content

# 取得右側資訊欄結果
def extract_rightInfo(text):
    right_info = re.findall(r'<div class="BNeawe s3v9rd AP7Wnd"><div><div class="BNeawe s3v9rd AP7Wnd">(?P<content>.*?)<span>', str(text), re.DOTALL)
    if right_info != []:
        right_info = right_info[0].replace('\n', '')
        right_info = re.sub(u'<span class="BNeawe">.*?<div class="BNeawe s3v9rd AP7Wnd">', "", right_info) 
    return right_info

def get_encyclopedia_ans(query):
    token = jieba_tokenizer(query)
    check_res = pos_checker(check_list, token)
    html = get_html(query, 1)
    
    if check_res:
        answer = extract_rightInfo(html)
    else:
        candidate = html.select('div.ZINbbc.xpd.O9g5cc.uUPGi')
        answer = get_content(candidate)
    return answer

# 3.聊天機器人answer selection

In [ ]:
# 請將 "3.聊天機器人answer selection" 的程式碼修改如下

In [ ]:
# jieba 詞性參考 => http://blog.pulipuli.info/2017/11/fasttag-identify-part-of-speech-in.html

def query_classifier(token):
    '''
    return
     True => samll talk
     False => encyclopedia
    '''
    count_nouns, count_verbs, count_advs, count_adjs = 0, 0, 0, 0

    token_len = len(token)
    for i, token in enumerate(token):
        word, pos = token

        if pos == "x": # 標點符號
            token_len -= 1
                      
        if any(pos == p for p in "n ng nr nrfg nrt ns nt nz".split(" ")): # 名詞類
            count_nouns += 1
        elif any(pos == p for p in "v vg vd vi vn vq".split(" ")): # 動詞類
            count_verbs += 1
        elif any(pos == p for p in "a ag ad an b".split(" ")): # 副詞類
            count_advs += 1
        elif any(pos == p for p in "d dg df e zg".split(" ")): # 形容詞類
            count_adjs += 1

    
    # 若完全沒比對出名詞 可能為口語短句 應忽略
    if count_nouns == 0:
        return True
     # 若比對出副詞但token長度 < 3 可能為口語短句 應忽略
    elif count_advs != 0 and token_len <= 3:
        return True
    # 若一個名詞 + 一個形容詞或副詞 判斷其僅為名詞的補充說明 應忽略
    elif count_nouns == 1 and (count_advs != 0 or count_adjs != 0):
        if token_len < 4:
            return True
        else:
            return False
     # 若動詞超過兩個 可能為非口語短句 不應忽略
    elif count_verbs >= 2:
        return False
    else:
        return False
    
def get_response(query):
    token = jieba_tokenizer(query)
    res = query_classifier(token)
    
    cutomize_ans = get_cutomize_ans(query)
    # 先判斷是否為自定義語句
    if cutomize_ans != None:
        return cutomize_ans
    else:
        # 閒聊問句
        if res:
            return get_api_ans(query)
        # 知識問句
        else:
            return get_encyclopedia_ans(query)   